In [53]:
from data_preprocess import *

In [54]:
source_vocab = Vocab()
target_vocab = Vocab()

# Build vocab from training data
with open(train_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        target, source, _ = line.strip().split('\t')
        source_vocab.add_string(source)
        target_vocab.add_string(target)

In [55]:
train_file_path = 'dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv'
train_dataset = TransliterationDataset(train_file_path, source_vocab, target_vocab)

In [59]:
path = train_file_path
data = []
with open(path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t') # read from tsv file and extract data
    c = 0
    for row in reader:
        if len(row) != 3:
            continue
        tgt, src, count = row # extract target devanagari script and source latin script
        src_ids = source_vocab.encode(src, add_sos_eos=False) # encode source sequence
        tgt_ids = target_vocab.encode(tgt, add_sos_eos=True) # encode target sequence
        for _ in range(int(count)):
            data.append((src_ids,tgt_ids))
# print(src+' -> '+tgt)
# print(src_ids)
# print(tgt_ids)
print(len(data))

84942


In [56]:
len(train_dataset.data)

44204

'agnashaya'